In [47]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import numpy as np
from random import choice
import random
import pandas as pd
import socket
import socks

In [45]:
def get_info(url):
    # Получение информции со страницы объявления
    page = requests.get(url,headers = choice(agents))
    #print(page.status_code)
    soup = BeautifulSoup(page.text,'html.parser')
    try:
        deal = re.split('[>;</]',str(soup.findAll(attrs={"data-marker":"item-description/title"})))[5]
        phone = re.findall('\d{11}',str(soup.findAll(attrs={"data-marker":"item-contact-bar/call"})))[0]
        text = str(soup.findChildren(attrs={'itemprop':'description'})).split('"')[1]
        flours = re.split('[>;</]',str(soup.findAll(attrs={'data-marker':'item-properties-item(8)/description'})))[3]
        material = re.split('[>;</]',str(soup.findAll(attrs={'data-marker':'item-properties-item(6)/description'})))[3]
        to_city = re.split('[>;</]',str(soup.findAll(attrs={'data-marker':'item-properties-item(5)/description'})))[3]
        addres = re.split('[>;</]',str(soup.findAll(attrs={'data-marker':'delivery/location'})))[3]
        price = re.split('[>;</]',str(soup.findAll(attrs={'data-marker':'item-description/price'})))[3]
    except:
        phone = 'No phone'
    return deal, phone, flours, material, to_city, addres, text, price

In [3]:
def get_links_from_page(url):
    # Получение ссылок на все объявления со страницы
    page = find_proxy(url)
    soup = BeautifulSoup(page.text,'html.parser')
    links=[]
    for a in soup.find_all(href=True,attrs={'class':"item-description-title-link"}):
        links.append(a['href'])
    
    return links

In [4]:
def get_proxy_list():
    # Получение списка прокси
    #https://www.proxy-list.download/HTTP
    proxy_list = open('Proxy List.txt','rt').read().split('\n')[1:-1]
    proxy=pd.DataFrame(data = proxy_list,columns=['proxy'])
    proxy['state'] = 1
    return proxy

In [5]:
def get_agents():
    # Получение списка user-agents
    agents_list = open('agents.txt','rt').read().split('\\\n')[1:-1]
    agents=[]
    for i in agents_list:
        agents.append({'User-agent' :  i})
    return agents

In [6]:
def find_proxy(url):
    # Поиск работоспособного прокси
    answer=0
    while answer != 200:
        try:
            hider = hide_generator()
            r = requests.get(url,headers = hider[1], proxies = {'http':'http://'+hider[0],'https':'http://'+hider[0]})
            answer=r.status_code
            print('Proxy ' + hider[0] + ' is OK!')
        except:
            print("Proxy " + hider[0] + " doesn't work!")
            proxies.loc[proxies[proxies['proxy']==hider[0]].index]=0
            continue
    return r

In [7]:
def hide_generator():
    # Генерация рандомной комбинации proxy-useragent
    return choice(proxies[proxies.state==1].proxy), choice(agents)

In [8]:
def get_last_page(url):
    # Поиск номера последней страницы
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'lxml')
    last_page = str(soup.find_all(attrs={'class':'pagination-page'},text='Последняя'))
    last = int(last_page.split('p=')[1].split('"')[0])
    return last

In [23]:
%%time

main_url = 'https://www.avito.ru/samara/doma_dachi_kottedzhi/prodam?s_trg=4&f=1248_0b14646'


proxies = get_proxy_list()
agents = get_agents()
links=[]
page_count = 48
services = []
all_pages = 51#get_last_page(main_url)
while page_count<=all_pages:
    if page_count ==1:
        page_url = main_url + 's_trg=4&f=1248_0b14646'
    else:
        page_url = main_url + '?p=' + str(page_count) + '&f=1248_0b14646'
    links.append(get_links_from_page(page_url))
    print('Page ' + str(page_count) + ' parsed')
    page_count+=1
    sleep(np.random.randint(1,3))
#out=[]
for i in links:
    for j in i:
        out.append(j)



Proxy 180.250.247.123:23500 doesn't work!
Proxy 102.135.176.234:8080 doesn't work!
Proxy 185.242.181.35:8080 doesn't work!
Proxy 5.141.86.233:50059 doesn't work!
Proxy 182.254.204.42:1080 doesn't work!
Proxy 191.102.70.82:999 doesn't work!
Proxy 134.119.205.173:1080 doesn't work!
Proxy 67.205.163.53:8080 doesn't work!
Proxy 157.230.232.130:3128 doesn't work!
Proxy 109.69.1.16:45827 is OK!
Page 48 parsed
Proxy 212.3.210.61:37344 doesn't work!
Proxy 117.196.236.235:52177 doesn't work!
Proxy 217.61.15.193:8080 doesn't work!
Proxy 51.254.124.179:3128 is OK!
Proxy 36.89.189.203:43305 doesn't work!
Proxy 190.248.153.162:8080 doesn't work!
Proxy 190.248.153.162:8080 doesn't work!
Proxy 170.247.29.138:8080 is OK!
Page 49 parsed
Proxy 103.127.95.26:45422 doesn't work!
Proxy 195.122.185.95:3128 doesn't work!
Proxy 80.211.45.34:80 doesn't work!
Proxy 86.123.166.109:8080 doesn't work!
Proxy 104.248.108.33:8080 doesn't work!
Proxy 87.228.103.111:8080 is OK!
Page 50 parsed
Proxy 1.196.160.79:9999 is

In [21]:
#out=[]
for i in links2:
    for j in i:
        out.append(j)

In [37]:
q=pd.DataFrame(out)

In [39]:
q.to_csv('q.csv',index=False)

In [52]:
%%time
#socks.set_default_proxy(socks.SOCKS5,'localhost',9150)
#socket.socket=socks.socksocket
agents = get_agents()
df=pd.DataFrame(columns=['deal', 'phone', 'flours', 'material', 'to_city', 'addres', 'text', 'price'])
for i, l in enumerate(out):
    try:
        info = get_info('https://m.avito.ru'+l)
        #services.append(info)
        df.loc[df.shape[0]]=info
        print(str(len(out)) + '/' + str(i+1))
        sleep(random.randint(2,5))
    except:
        print('qqq')
        sleep(10)
        continue
#df=pd.DataFrame(data=services)
#df.to_csv('remont_tech.csv',encoding='UTF-8',index=False)

2646/1
2646/2
2646/3
2646/4
2646/5
2646/6
2646/7
2646/8
2646/9
2646/10
2646/11
2646/12
2646/13
2646/14
2646/15
2646/16
2646/17
2646/18
2646/19
2646/20
2646/21
2646/22
2646/23
2646/24
2646/25
2646/26
2646/27
2646/28
2646/29
2646/30
2646/31
2646/32
2646/33
2646/34
2646/35
2646/36
2646/37
2646/38
2646/39
2646/40
2646/41
2646/42
2646/43
2646/44
2646/45
2646/46
2646/47
2646/48
2646/49
2646/50
2646/51
2646/52
2646/53
2646/54
2646/55
2646/56
2646/57
2646/58
2646/59
2646/60
2646/61
2646/62
2646/63
2646/64
2646/65
2646/66
2646/67
2646/68
2646/69
2646/70
2646/71
2646/72
2646/73
2646/74
2646/75
2646/76
2646/77
2646/78
2646/79
2646/80
2646/81
2646/82
2646/83
2646/84
2646/85
2646/86
2646/87
2646/88
2646/89
2646/90
2646/91
2646/92
2646/93
2646/94
2646/95
2646/96
2646/97
2646/98
2646/99
2646/100
2646/101
2646/102
2646/103
2646/104
2646/105
2646/106
2646/107
2646/108
2646/109
2646/110
2646/111
2646/112
2646/113
2646/114
2646/115
2646/116
2646/117
2646/118
2646/119
2646/120
2646/121
2646/122
2646/123
2

2646/925
2646/926
2646/927
2646/928
2646/929
2646/930
2646/931
2646/932
2646/933
2646/934
2646/935
2646/936
2646/937
2646/938
2646/939
2646/940
2646/941
2646/942
2646/943
2646/944
2646/945
2646/946
2646/947
2646/948
2646/949
2646/950
2646/951
2646/952
2646/953
2646/954
2646/955
2646/956
2646/957
2646/958
2646/959
2646/960
2646/961
2646/962
2646/963
2646/964
2646/965
2646/966
2646/967
2646/968
2646/969
2646/970
2646/971
2646/972
2646/973
2646/974
2646/975
2646/976
2646/977
2646/978
2646/979
2646/980
2646/981
2646/982
2646/983
2646/984
2646/985
2646/986
2646/987
2646/988
2646/989
2646/990
2646/991
2646/992
2646/993
2646/994
2646/995
2646/996
2646/997
2646/998
2646/999
2646/1000
2646/1001
2646/1002
2646/1003
2646/1004
2646/1005
2646/1006
2646/1007
2646/1008
2646/1009
2646/1010
2646/1011
2646/1012
2646/1013
2646/1014
2646/1015
2646/1016
2646/1017
2646/1018
2646/1019
2646/1020
2646/1021
2646/1022
2646/1023
2646/1024
2646/1025
2646/1026
2646/1027
2646/1028
2646/1029
2646/1030
2646/1031
2646/

2646/1753
2646/1754
2646/1755
2646/1756
2646/1757
2646/1758
2646/1759
2646/1760
2646/1761
2646/1762
2646/1763
2646/1764
2646/1765
2646/1766
2646/1767
2646/1768
2646/1769
2646/1770
2646/1771
2646/1772
2646/1773
2646/1774
2646/1775
2646/1776
2646/1777
2646/1778
2646/1779
2646/1780
2646/1781
2646/1782
2646/1783
2646/1784
2646/1785
2646/1786
2646/1787
2646/1788
2646/1789
2646/1790
2646/1791
2646/1792
2646/1793
2646/1794
2646/1795
2646/1796
2646/1797
2646/1798
2646/1799
2646/1800
2646/1801
2646/1802
2646/1803
2646/1804
2646/1805
2646/1806
2646/1807
2646/1808
2646/1809
2646/1810
2646/1811
qqq
2646/1813
2646/1814
2646/1815
2646/1816
2646/1817
2646/1818
2646/1819
2646/1820
2646/1821
2646/1822
2646/1823
2646/1824
2646/1825
2646/1826
2646/1827
2646/1828
2646/1829
2646/1830
2646/1831
2646/1832
2646/1833
2646/1834
2646/1835
2646/1836
2646/1837
2646/1838
2646/1839
2646/1840
2646/1841
2646/1842
2646/1843
2646/1844
2646/1845
2646/1846
2646/1847
2646/1848
2646/1849
2646/1850
2646/1851
2646/1852
2646/1

2646/2576
2646/2577
2646/2578
2646/2579
2646/2580
2646/2581
2646/2582
2646/2583
2646/2584
2646/2585
2646/2586
2646/2587
2646/2588
2646/2589
2646/2590
2646/2591
2646/2592
2646/2593
2646/2594
2646/2595
2646/2596
2646/2597
2646/2598
2646/2599
2646/2600
2646/2601
2646/2602
2646/2603
2646/2604
2646/2605
2646/2606
2646/2607
2646/2608
2646/2609
2646/2610
2646/2611
2646/2612
2646/2613
2646/2614
2646/2615
2646/2616
2646/2617
2646/2618
2646/2619
2646/2620
2646/2621
2646/2622
2646/2623
2646/2624
2646/2625
2646/2626
2646/2627
2646/2628
2646/2629
2646/2630
2646/2631
2646/2632
2646/2633
2646/2634
2646/2635
2646/2636
2646/2637
2646/2638
2646/2639
2646/2640
2646/2641
2646/2642
2646/2643
2646/2644
2646/2645
2646/2646
CPU times: user 2min 40s, sys: 8.87 s, total: 2min 49s
Wall time: 3h 56min 32s


In [56]:
df.to_csv('house.csv',sep='$')